In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv('./files/video_statistics.csv',index_col=0)

In [7]:
import re

channel_names = data['channelname'].unique()

tag_warehouse=[]

for channel_name in channel_names:
    tags = ','.join(list(data[data['channelname'] == channel_name]['tags']))
    tags = set(tags.split(','))
    sentence = ' '.join(tags)
    sentence = re.findall(r'[가-힣]+',sentence)
    
    tag_warehouse.append(sentence)
    
print(len(tag_warehouse))
    
    

In [ ]:
from konlpy.tag import Okt
from collections import Counter

JVM_PATH = '/Library/Java/JavaVirtualMachines/zulu-15.jdk/Contents/Home/bin/java'


okt = Okt(jvmpath=JVM_PATH)


# noun = okt.nouns(' '.join(tag_warehouse))
# count = Counter(noun)

# noun_list = count.most_common(100)
# for v in noun_list:
#     print(v)


In [57]:
import pandas as pd


In [93]:
data = pd.read_csv('files/matrix/먹방_views.csv',index_col=0)

In [94]:
channelid = data['channelid']
data.drop('channelid',axis=1,inplace=True)

In [95]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

In [96]:
data = data.T
data.head()

,ENA 이엔에이,Byungari 병아리언니,수빙수tv sooBingsoo,주식타이밍,운동부 둘이 왔어요,분요리 뚝딱이형,상해기SangHyuk,전승배,끼룩푸드 seagull food,미자네 주막,...,sini시니,지유개 zziU & Ezzi,홍현희 제이쓴의 홍쓴TV,히밥heebab,광마니,흥삼이네 Heungsam's Family,집나간햄지 Wandering Hamzy,곽준빈의 세계기사식당,꼬꼬PD가족[Kkokko PD Family],어서와 한국은 처음이지
떡볶이,0.0,0.0,0.000000,0.0,9.767972e+05,210953.500000,412739.307692,0.000000,2.111251e+06,5.378080e+05,...,255703.200000,379699.000000,926037.666667,659708.750000,560956.421053,6.019942e+05,396736.000000,0.000000,285056.777778,19285.500000
치즈,0.0,0.0,637468.500000,0.0,1.064184e+06,162503.000000,467402.244444,0.000000,1.444262e+06,0.000000e+00,...,251293.612245,90805.333333,0.000000,639870.000000,709285.250000,8.896753e+05,477878.000000,0.000000,263723.000000,11448.000000
먹방,0.0,0.0,585693.870968,0.0,1.008243e+06,0.000000,405236.824121,297296.714286,0.000000e+00,8.484311e+05,...,249409.395000,328300.937500,926247.000000,655636.982857,715548.655000,1.242305e+06,587451.109375,352591.363636,283038.855000,18414.490909
음식,0.0,0.0,424208.111111,464446.0,9.610186e+05,243641.647059,364364.000000,155480.714286,1.067064e+06,8.015585e+05,...,247231.500000,180847.000000,660627.750000,708409.320000,578119.200000,1.364514e+06,559111.376147,339515.259259,304099.380952,15672.230769
김치,0.0,0.0,594192.000000,0.0,1.026367e+06,146101.000000,407409.952381,364942.750000,5.922617e+05,1.118122e+06,...,266293.642857,106001.000000,813646.000000,446708.666667,651658.882353,1.449040e+06,573741.166667,0.000000,295880.783133,0.000000


In [97]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = data.values
print(matrix)
# view_mean은 keyword의 평균 조회수
view_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-영화에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - view_mean.reshape(-1, 1)

[[     0.              0.              0.         ...      0.
  285056.77777778  19285.5       ]
 [     0.              0.         637468.5        ...      0.
  263723.          11448.        ]
 [     0.              0.         585693.87096774 ... 352591.36363636
  283038.855       18414.49090909]
 ...
 [     0.              0.         768657.         ...      0.
       0.              0.        ]
 [     0.              0.              0.         ...      0.
  277936.5             0.        ]
 [     0.              0.              0.         ...      0.
  532789.5             0.        ]]


In [98]:
matrix.shape

(100, 80)

In [99]:
pd.DataFrame(matrix_user_mean,columns=data.columns,index=data.index).head()

,ENA 이엔에이,Byungari 병아리언니,수빙수tv sooBingsoo,주식타이밍,운동부 둘이 왔어요,분요리 뚝딱이형,상해기SangHyuk,전승배,끼룩푸드 seagull food,미자네 주막,...,sini시니,지유개 zziU & Ezzi,홍현희 제이쓴의 홍쓴TV,히밥heebab,광마니,흥삼이네 Heungsam's Family,집나간햄지 Wandering Hamzy,곽준빈의 세계기사식당,꼬꼬PD가족[Kkokko PD Family],어서와 한국은 처음이지
떡볶이,-686746.481863,-686746.481863,-686746.481863,-686746.481863,290050.768137,-475792.981863,-274007.174171,-686746.481863,1.424505e+06,-148938.481863,...,-431043.281863,-307047.481863,239291.184804,-27037.731863,-125790.060810,-84752.231863,-290010.481863,-686746.481863,-401689.704085,-667460.981863
치즈,-605164.240186,-605164.240186,32304.259814,-605164.240186,459019.759814,-442661.240186,-137761.995741,-605164.240186,8.390979e+05,-605164.240186,...,-353870.627941,-514358.906853,-605164.240186,34705.759814,104121.009814,284511.093147,-127286.240186,-605164.240186,-341441.240186,-593716.240186
먹방,-826625.819854,-826625.819854,-240931.948886,-826625.819854,181617.212933,-826625.819854,-421388.995734,-529329.105568,-8.266258e+05,21805.240752,...,-577216.424854,-498324.882354,99621.180146,-170988.836997,-111077.164854,415678.860146,-239174.710479,-474034.456218,-543586.964854,-808211.328945
음식,-671945.370803,-671945.370803,-247737.259692,-207499.370803,289073.243876,-428303.723744,-307581.370803,-516464.656517,3.951190e+05,129613.129197,...,-424713.870803,-491098.370803,-11317.620803,36463.949197,-93826.170803,692568.129197,-112833.994656,-332430.111543,-367845.989850,-656273.140034
김치,-748694.284875,-748694.284875,-154502.284875,-748694.284875,277673.039199,-602593.284875,-341284.332495,-383751.534875,-1.564326e+05,369427.881791,...,-482400.642018,-642693.284875,64951.715125,-301985.618209,-97035.402523,700345.715125,-174953.118209,-748694.284875,-452813.501743,-748694.284875


In [100]:
U, sigma, Vt = svds(matrix_user_mean, k = 12)
sigma = np.diag(sigma)
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + view_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = data.columns,index=data.index)
df_svd_preds.head()



,ENA 이엔에이,Byungari 병아리언니,수빙수tv sooBingsoo,주식타이밍,운동부 둘이 왔어요,분요리 뚝딱이형,상해기SangHyuk,전승배,끼룩푸드 seagull food,미자네 주막,...,sini시니,지유개 zziU & Ezzi,홍현희 제이쓴의 홍쓴TV,히밥heebab,광마니,흥삼이네 Heungsam's Family,집나간햄지 Wandering Hamzy,곽준빈의 세계기사식당,꼬꼬PD가족[Kkokko PD Family],어서와 한국은 처음이지
떡볶이,46152.828542,47678.765047,426781.530209,65346.844862,6.360183e+05,164570.780691,403073.239192,202541.573699,2.088875e+06,166009.943280,...,259304.163224,245705.987668,1.067281e+06,734028.101011,616079.279837,6.754467e+05,470014.301492,68300.318757,295625.786968,59974.760557
치즈,74998.897707,74011.069602,352048.369450,79072.831161,5.813091e+05,158302.101247,432327.734295,137409.938981,1.488058e+06,261132.552587,...,287827.971494,167885.723575,3.630427e+05,400183.759251,589281.302923,8.902596e+05,345140.685795,68198.677634,306496.081178,87226.157283
먹방,93227.053282,96095.123527,795193.947046,95794.290944,9.618463e+05,169403.206477,502592.708075,373947.653769,-2.800579e+04,963515.379153,...,335214.881421,273118.555233,9.968608e+05,640903.383641,691127.718730,9.522393e+05,585351.803049,294737.421329,360992.411790,121487.782330
음식,44828.180100,44463.304275,312398.199620,82212.142878,7.402489e+05,200858.798586,413050.103209,217010.084283,1.098838e+06,610451.933789,...,272375.088074,181310.211976,6.564043e+05,767305.414433,741608.350407,1.013378e+06,555679.843027,110758.805788,374242.474795,64936.381657
김치,26086.601002,27565.271408,618684.373062,29924.322746,1.017088e+06,116237.860456,459769.501172,265567.848330,5.126965e+05,906573.736626,...,303256.322686,185570.372428,7.635549e+05,542079.765696,708346.060260,1.377414e+06,577311.477824,159244.617089,339152.521021,42253.801362


In [101]:
df_svd_preds.iloc[0].sort_values()

재밌는 거 올라온다        -1.058875e+05
조나단               -1.058875e+05
THE 트롯 보라고         4.519834e+04
ENA 이엔에이           4.615283e+04
Byungari 병아리언니     4.767877e+04
                       ...     
tzuyang쯔양          2.798581e+06
HIU 하이유            3.288962e+06
보경 Bokyoung        3.711377e+06
[햄지]Hamzy          4.174749e+06
푸먹_foomuk          5.924344e+06
Name: 떡볶이, Length: 80, dtype: float64

In [109]:
df_svd_preds = df_svd_preds.T
df_svd_preds['channedid'] = channelid

In [110]:
df_svd_preds

,떡볶이,치즈,먹방,음식,김치,라면,튀김,불닭볶음면,김밥,양념,...,통닭,쫄면,카레,간장,가지,탕수육,소세지,김치찌개,한식,channedid
ENA 이엔에이,46152.828542,74998.897707,93227.053282,4.482818e+04,2.608660e+04,83522.402160,-2.430409e+04,7219.636733,-12079.761787,4.777915e+04,...,-2.397992e+04,-1426.243120,-90918.415518,-1.135040e+04,-7.319110e+03,8096.695495,-8.384972e+04,-9.868003e+04,-5.014050e+03,UCAP8OK0GHFbnL5OkY3j04Xw
Byungari 병아리언니,47678.765047,74011.069602,96095.123527,4.446330e+04,2.756527e+04,82800.993622,-2.641286e+04,9677.160467,-10158.960343,4.868114e+04,...,-2.259835e+04,-6570.906726,-98850.067740,-1.454361e+04,-4.480194e+03,7247.568172,-8.590562e+04,-9.966847e+04,-7.513397e+03,UC8TCA15kPzNw9ppRaVZ937w
수빙수tv sooBingsoo,426781.530209,352048.369450,795193.947046,3.123982e+05,6.186844e+05,396666.409344,3.025712e+05,360900.427395,495380.275258,4.175057e+05,...,3.136504e+05,185507.637002,210269.522838,3.863076e+05,3.183044e+05,267886.276811,4.398431e+05,2.482223e+05,2.252714e+05,UCIUfR-2qcpWqgAIfZJXOLWA
주식타이밍,65346.844862,79072.831161,95794.290944,8.221214e+04,2.992432e+04,103020.309683,-1.381458e+04,14005.479826,10635.943792,4.535483e+04,...,-2.269414e+04,9248.569667,-79475.715663,-1.171817e+04,-6.085704e+03,23694.172114,-9.493165e+04,-9.491678e+04,6.945350e+03,UCvil4OAt-zShzkKHsg9EQAw
운동부 둘이 왔어요,636018.256648,581309.134857,961846.348612,7.402489e+05,1.017088e+06,774834.430005,7.299001e+05,587649.974880,780065.162682,7.324207e+05,...,5.904940e+05,420318.763498,692372.663869,1.058754e+06,5.884512e+05,768694.013907,4.510516e+05,9.035531e+05,7.875158e+05,UC__J0wvxgqq4kg3i5iTgkLQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
흥삼이네 Heungsam's Family,675446.749290,890259.632086,952239.274497,1.013378e+06,1.377414e+06,999819.217996,1.262019e+06,854414.190190,493949.200842,1.494293e+06,...,1.333321e+06,586627.987699,529697.502523,1.863379e+06,1.178702e+06,862589.158006,1.782069e+06,1.420318e+06,1.352531e+06,UC5XuQ-xiWAB6f-qu6gJMDBQ
집나간햄지 Wandering Hamzy,470014.301492,345140.685795,585351.803049,5.556798e+05,5.773115e+05,421008.599618,3.834430e+05,272333.465943,507271.180186,4.142570e+05,...,5.127641e+05,380613.757413,386509.466822,6.351770e+05,4.566853e+05,532596.903237,5.690076e+05,4.391227e+05,5.696287e+05,UC1EiHIhV6UDRWOY6BaEavdw
곽준빈의 세계기사식당,68300.318757,68198.677634,294737.421329,1.107588e+05,1.592446e+05,112870.577670,-8.607693e+02,22567.062228,26132.541584,3.881691e+04,...,-2.992186e+04,11508.968372,-54302.326969,5.230384e+03,9.287682e+03,34066.390168,-6.887999e+04,-6.369326e+04,9.159717e+03,UCL7BB2StsDBYE2kU1a6127g
꼬꼬PD가족[Kkokko PD Family],295625.786968,306496.081178,360992.411790,3.742425e+05,3.391525e+05,358775.896898,2.699904e+05,225126.524871,309230.046830,3.403339e+05,...,2.827624e+05,259620.938301,238971.929880,3.440524e+05,2.557260e+05,261204.723858,2.023605e+05,2.211978e+05,3.038027e+05,UCfugxK22rRKUWMtgYNTvzog


In [ ]:
import pandas as pd

data = pd.read_csv('../files/full_matrix/먹방_views_fullmatrix.csv',index_col=0)
data1 = pd.read_csv('../files/full_matrix/먹방_likes_fullmatrix.csv',index_col=0)
data2 = pd.read_csv('../files/full_matrix/먹방_comments_fullmatrix.csv',index_col=0)

data

In [26]:
data = pd.read_csv('../files/full_matrix/먹방_views_fullmatrix.csv',index_col=0)
# data.loc[data<0]  =0
# data


TypeError: '<' not supported between instances of 'str' and 'int'

In [20]:
column = '떡볶이'
data.sort_values(by=column,ascending=False).head(20)

,떡볶이,치즈,먹방,음식,김치,라면,튀김,불닭볶음면,김밥,양념,...,통닭,쫄면,카레,간장,가지,탕수육,소세지,김치찌개,한식,channedid
푸먹_foomuk,5.924344e+06,7.052732e+06,7.302770e+06,7.268890e+06,7.924099e+06,7.567847e+06,8.828252e+06,5.962582e+06,7.580357e+06,7.990994e+06,...,6.623931e+06,9.196296e+06,9.380061e+06,1.041715e+07,7.215670e+06,7.773759e+06,7.265819e+06,1.304203e+07,8.510414e+06,UCOszRhGW_mW8yPYKhLuULMw
[햄지]Hamzy,4.174749e+06,4.614794e+06,4.424695e+06,4.240177e+06,4.472996e+06,4.466663e+06,3.796458e+06,3.561425e+06,5.634759e+06,4.610969e+06,...,6.200836e+06,4.423833e+06,3.013160e+06,5.346868e+06,5.889950e+06,3.026947e+06,7.994588e+06,4.399941e+06,3.369847e+06,UCPKNKldggioffXPkSmjs5lQ
보경 Bokyoung,3.711377e+06,7.450051e+05,1.598325e+06,9.194886e+05,7.912920e+05,9.106307e+05,8.886520e+05,1.176683e+06,1.099790e+06,8.233725e+05,...,1.583833e+06,5.610922e+05,5.735294e+05,9.197957e+05,6.562246e+05,5.525290e+05,1.426293e+07,5.129561e+05,8.623753e+05,UCALl6UrYtJndCXo2p6hANPQ
HIU 하이유,3.288962e+06,1.780076e+06,2.864309e+06,3.676818e+06,2.077167e+06,3.577200e+06,1.970918e+06,3.313946e+06,3.279671e+06,1.418304e+06,...,8.122607e+04,6.176944e+05,1.353358e+06,6.317797e+04,9.460106e+05,1.310810e+06,1.932710e+06,3.118700e+06,6.516405e+05,UCw-QmownJwilhTje10TYlIg
tzuyang쯔양,2.798581e+06,2.203756e+06,3.121625e+06,2.950975e+06,2.729232e+06,3.074871e+06,2.785465e+06,1.122017e+05,2.671299e+06,2.607677e+06,...,1.376322e+06,4.312032e+06,1.098787e+07,2.702900e+06,-4.433357e+04,3.176468e+06,-8.072204e+04,2.264299e+06,4.384118e+06,UCfpaSruWW3S4dibonKXENjA
설기양SULGI,2.239993e+06,2.238169e+06,2.638528e+06,5.264292e+05,1.986767e+06,2.692458e+06,2.077809e+06,3.300095e+06,1.443330e+06,2.888904e+06,...,2.510271e+05,1.847776e+05,1.173142e+06,8.331017e+05,9.889011e+05,1.838449e+05,4.005580e+06,3.558747e+06,1.358997e+05,UCIafb9ExBkh4WsCOagZssZA
끼룩푸드 seagull food,2.088875e+06,1.488058e+06,-2.800579e+04,1.098838e+06,5.126965e+05,2.733719e+06,1.619535e+06,-5.342765e+04,1.327990e+06,1.310550e+06,...,2.291945e+05,2.466923e+04,2.098527e+05,1.946129e+06,3.948327e+05,1.648425e+06,9.998238e+03,5.194895e+04,8.755023e+05,UCa0aTySoJBHXpHigv5tsxbQ
문복희 Eat with Boki,2.045716e+06,2.123420e+06,1.723933e+06,1.918060e+06,1.958565e+06,2.364421e+06,2.106137e+06,2.071032e+06,2.365047e+06,2.448395e+06,...,1.998971e+06,1.565729e+06,1.929090e+06,2.255101e+06,1.825924e+06,1.344472e+06,2.707796e+06,2.271581e+06,1.602336e+06,UCoLQZ4ZClFqVPCvvjuiUSRA
쏘영 Ssoyoung,1.984017e+06,1.315308e+06,2.459110e+06,1.356759e+06,1.674907e+06,1.320625e+06,8.959342e+05,1.650847e+06,1.802519e+06,2.282012e+06,...,1.987754e+06,3.855444e+05,1.191449e+03,-5.089519e+04,1.732102e+06,-2.927880e+04,2.498156e+06,-4.593104e+04,2.884422e+05,UC4PpFUrfT2Pou7OwpVF0MUQ
[상윤쓰]Sangyoon,1.458087e+06,1.527374e+06,1.590059e+06,1.472159e+06,1.993047e+06,2.050660e+06,1.759476e+06,2.135412e+06,2.105266e+06,1.885324e+06,...,1.030450e+06,1.277779e+05,3.573947e+05,1.899444e+06,1.134173e+06,9.822044e+05,4.477454e+05,1.655945e+06,8.111617e+05,UCb-y7YXK75lcU76lyFwBDgQ


In [22]:
column = '떡볶이'

chan_dict = dict(data[column])
id_dict = dict(data['channedid'])

In [24]:
a=[]
a.append([chan_dict,id_dict])
a.append([chan_dict,id_dict])
a.append([chan_dict,id_dict])
a

[[{'ENA 이엔에이': 46152.828541742056,
   'Byungari 병아리언니 ': 47678.765046797576,
   '수빙수tv sooBingsoo': 426781.5302088434,
   '주식타이밍': 65346.84486239089,
   '운동부 둘이 왔어요': 636018.2566481925,
   '분요리 뚝딱이형': 164570.7806914145,
   '상해기SangHyuk': 403073.2391919826,
   '전승배': 202541.5736994221,
   '끼룩푸드 seagull food': 2088875.358746472,
   '미자네 주막': 166009.94327954587,
   '리얼깽TV RealkkaengTV': 177891.14882270247,
   'hongsi 홍시': 540867.1784973515,
   '혼술다큐 미짱 lonely mizzang': 903292.1246423152,
   '이연복의 복주머니': 88996.75958363153,
   '문복희 Eat with Boki': 2045716.019693154,
   '대평이 Daepyeong': 193752.33174648648,
   '푸메Fume': 578386.9227483226,
   'tzuyang쯔양': 2798581.217702792,
   '[상윤쓰]Sangyoon': 1458086.850855332,
   '까니짱 [ G-NI : 蟹ちゃん]': 819129.8793263908,
   '가오니의 메뉴판GAONI': 468304.9119634994,
   '조나단': -105887.51182439004,
   '삼대장 Samdaejang': 301556.9199153532,
   '설기양SULGI': 2239992.5957363853,
   '바다곰 조해웅Seabear': 155749.9585251488,
   'LG헬로비전 헬로tv': 49973.472614793456,
   'SIO ASMR': 1312

In [25]:
a[0]

[{'ENA 이엔에이': 46152.828541742056,
  'Byungari 병아리언니 ': 47678.765046797576,
  '수빙수tv sooBingsoo': 426781.5302088434,
  '주식타이밍': 65346.84486239089,
  '운동부 둘이 왔어요': 636018.2566481925,
  '분요리 뚝딱이형': 164570.7806914145,
  '상해기SangHyuk': 403073.2391919826,
  '전승배': 202541.5736994221,
  '끼룩푸드 seagull food': 2088875.358746472,
  '미자네 주막': 166009.94327954587,
  '리얼깽TV RealkkaengTV': 177891.14882270247,
  'hongsi 홍시': 540867.1784973515,
  '혼술다큐 미짱 lonely mizzang': 903292.1246423152,
  '이연복의 복주머니': 88996.75958363153,
  '문복희 Eat with Boki': 2045716.019693154,
  '대평이 Daepyeong': 193752.33174648648,
  '푸메Fume': 578386.9227483226,
  'tzuyang쯔양': 2798581.217702792,
  '[상윤쓰]Sangyoon': 1458086.850855332,
  '까니짱 [ G-NI : 蟹ちゃん]': 819129.8793263908,
  '가오니의 메뉴판GAONI': 468304.9119634994,
  '조나단': -105887.51182439004,
  '삼대장 Samdaejang': 301556.9199153532,
  '설기양SULGI': 2239992.5957363853,
  '바다곰 조해웅Seabear': 155749.9585251488,
  'LG헬로비전 헬로tv': 49973.472614793456,
  'SIO ASMR': 1312794.4065855118,
  '애정 Aejeo